In [1]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data locations
raw_data_path = '../data/peaks_data'
outbase = '../output/peaks_data'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase

# training hyperparameters
batch_size = 50
num_batches_in_train = int(41305 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [3]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

reload(fxns)
print(test_out_path)
(seqt, labelt), info = fxns.get_seq_and_label(test_out_path, num_epochs=None)
seqt_batch, labelt_batch = tf.train.shuffle_batch([seqt, labelt],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

inputs are of size 1200
outputs are of size 12
../output/peaks_data/texttest


/anaconda2/lib/python2.7/site-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [4]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import novel_model; reload(novel_model)
modelname='peaks_freeconv'
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = novel_model.get_novel_model(info['seq_len'], info['label_len'], conv_infos)

150.0
75.0
38.0
1200 h 4 38.0


In [5]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [6]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver(max_to_keep=None)
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.7337293291091919
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.6688433718681336
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.60829052845637
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.5480066582560539
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.49955468916893003
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.46433893382549285
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.43836808596338545
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.41879790619015694
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.4034225683742099
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.39114264070987703
INFO:tensorflow:log/peaks_freeconv-250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.39114264070987703
Epoch: 0

In [7]:
#
# get test data

global_step_to_load_test = 1652


scores = []
labels = []

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=None)
    saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load_test))
    
    idx = 0
    while idx < 4348 / batch_size:
        batch_Xt, batch_Yt = sess.run([seqt_batch, labelt_batch])
        batch_logits = sess.run(logits, feed_dict={X: batch_Xt, Y: batch_Yt})
        scores.append(batch_logits)
        labels.append(batch_Yt)
        #print('asdf')
        idx += 1
        if idx % 100 == 0:
            print(idx)

scores_arr = np.concatenate(scores, axis=0)
labels_arr = np.concatenate(labels, axis=0)



INFO:tensorflow:Restoring parameters from log/peaks_freeconv-1652
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


In [8]:
accuracies = np.mean((scores_arr > 0).astype(int) == labels_arr.astype(int), axis=0)

print('accuracies:')
print(accuracies)

import sklearn.metrics
print('aucs:')
aucs = [sklearn.metrics.roc_auc_score(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(aucs)

import sklearn.metrics
def prec_recall(ys_true, ys_hat):
    ys, xs, thresholds = sklearn.metrics.precision_recall_curve(ys_true, ys_hat)
    return sklearn.metrics.auc(xs, ys, reorder=True)

print('auprcs:')
auprcs = [prec_recall(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(auprcs)

props = np.mean(labels_arr, axis=0)

import json
ids = sorted(json.loads(info['tf_to_pos'].replace("'", '"')).keys())

from IPython.display import display_pretty, display_html
import pandas as pd
results = pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs')
display_html(results.to_html(), raw=True)
results.to_csv('stats.' + modelname + '.tsv', index_label='id', sep='\t')

accuracies:
[0.89172414 0.90390805 0.89586207 0.90413793 0.90804598 0.91425287
 0.92643678 0.9783908  0.9062069  0.89448276 0.97494253 0.9016092 ]
aucs:
[0.5014162491810386, 0.4585547002389911, 0.5097870609700902, 0.4677719304342948, 0.49903132911392406, 0.4692241784361958, 0.5068300248138958, 0.5840315549512078, 0.48581235512977383, 0.5179529991841963, 0.5218541585094393, 0.5111955325101155]
auprcs:
[0.11338166161502039, 0.08765536101650141, 0.11186953503356162, 0.0864128939148339, 0.09519671171816435, 0.07999396285557314, 0.07092202124234837, 0.08737748290671762, 0.08857720648459849, 0.1126983231641521, 0.03636835333317742, 0.09775312492652447]


,acc,aucs,auprcs,props
FOS,0.903908,0.458555,0.087655,0.096092
JUN,0.904138,0.467772,0.086413,0.095862
MYC,0.914253,0.469224,0.079994,0.085747
PAX5,0.906207,0.485812,0.088577,0.093793
MAX,0.908046,0.499031,0.095197,0.091954
CTCF,0.891724,0.501416,0.113382,0.108276
NFKB1,0.926437,0.506830,0.070922,0.073563
GATA2,0.895862,0.509787,0.111870,0.104138
TBP,0.901609,0.511196,0.097753,0.098391
SPI1,0.894483,0.517953,0.112698,0.105517
